In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed, retries
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


install_requirements()
clear_output()
print("🚀 Setup complete. Continue to the next cell.")

In [ ]:
from dotenv import load_dotenv


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True, dotenv_path=".env")

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)

    print("Environment variables are set.")


setup_env()

In [ ]:
import os
import openai
from pydantic import BaseModel
import traceback
from typing import Union
import json

DEFAULT_OPENAI_MODEL = "gpt-4o"
DEFAULT_SYSTEM_PROMPT = "You are an intelligent AI assistant. The user will give you a prompt, respond appropriately."
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 1024


class EasyLLM:
    """
    A simple abstraction for the OpenAI API. It provides easy-to-use methods to generate text and objects using the OpenAI API.
    A demonstration for the "How to build an Abstaction with Open AI API" blog post.
    Author: Aditya Patange (AdiPat)
    """

    def __init__(
        self,
        api_key=os.getenv("OPENAI_API_KEY"),
        model=DEFAULT_OPENAI_MODEL,
        verbose=True,
        debug=True,
    ):
        self.verbose = verbose
        self.debug = debug

        if self.verbose:
            print("EasyLLM: Powering up! 🚀")

        self.api_key = api_key
        self.openai = openai.OpenAI(api_key=api_key)
        self.model = model

        if self.verbose:
            print(f"EasyLLM: Model set to {model}.")
            print("EasyLLM: Ready for some Generative AI action! ⚡️")

    def generate_text(
        self,
        prompt: str,
        system=DEFAULT_SYSTEM_PROMPT,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
    ) -> Union[str, None]:
        """Generates text using the OpenAI API."""
        try:
            if self.verbose or self.debug:
                print(f"Generating text for prompt: {prompt}")

            if self.debug:
                params = {
                    "prompt": prompt,
                    "system": system,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "model": self.model,
                }
                params = json.dumps(params, indent=2)
                print(f"Params: {params}")
            response = self.openai.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt},
                ],
                temperature=temperature,
                max_tokens=max_tokens,
            )

            response = response.choices[0].message.content

            if self.verbose or self.debug:
                print("Text generated successfully. 🎉")

            if self.debug:
                response = json.dumps(response)
                print(f"EasyLLM Response: {response}")
            return response
        except Exception as e:
            print(f"Failed to generate text. Error: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def generate_object(
        self,
        prompt: str,
        response_model: BaseModel,
        system=DEFAULT_SYSTEM_PROMPT,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
    ) -> Union[BaseModel, None]:
        """Generates an object using the OpenAI API and given response model."""
        try:
            if self.verbose or self.debug:
                print(f"Generating object for prompt: {prompt}")

            if self.debug:
                params = {
                    "prompt": prompt,
                    "system": system,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "model": self.model,
                }
                params = json.dumps(params, indent=2)
                print(f"Params: {params}")

            response = self.openai.beta.chat.completions.parse(
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt},
                ],
                response_format=response_model,
                model=self.model,
                temperature=temperature,
                max_tokens=max_tokens,
            )

            if self.verbose or self.debug:
                print("Object generated successfully. 🎉")

            if self.debug:
                response_json = response.model_dump_json()
                print(f"EasyLLM Response: {response_json}")
            return response.choices[0].message.parsed
        except Exception as e:
            print(f"Failed to generate object. Error: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def get_model(self) -> str:
        """Gets the current model."""
        return self.model

    def set_model(self, model: str) -> None:
        """Sets the model to the given model."""
        try:
            if self.verbose or self.debug:
                print(f"Setting model to {model}")
            self.openai = openai.OpenAI(api_key=self.api_key)
            self.model = model
            if self.verbose or self.debug:
                print(f"Model set to {model}")
        except Exception as e:
            print(f"Failed to set model.\nError: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

In [ ]:
import traceback

easyllm = EasyLLM()

DEFAULT_PRD_TOKENS = 8192

def generate_initial_prd(prompt: str) -> str:
    initial_prompt = f"Generate a Product Requirement Document for: '{prompt}'"
    system = "You are a Product Manager. The engineering team is waiting for a PRD for a new feature. Respond in markdown."
    return easyllm.generate_text(initial_prompt, system, max_tokens=DEFAULT_PRD_TOKENS)


def generate_prd_questions(prompt: str, prd: str) -> str:
    questions_prompt = f"Given a PRD for '{prompt}', generate a set of questions that can be asked to further refine the PRD.\nPRD: '{prd}'"
    return easyllm.generate_text(questions_prompt, max_tokens=DEFAULT_PRD_TOKENS // 2)


def generate_prd_answers(prompt: str, prd: str, questions: str) -> str:
    answers_prompt = f"Given a PRD for '{prompt}', and some questions: '{questions}', generate a set of answers to the questions.\nPRD: '{prd}'"
    return easyllm.generate_text(answers_prompt, max_tokens=DEFAULT_PRD_TOKENS // 2)


def generate_final_prd(prompt: str, prd: str, questions: str, answers: str) -> str:
    final_prd_prompt = f"Given a PRD for '{prompt}', and some questions: '{questions}', and answers: '{answers}', generate a final PRD.\nPRD: '{prd}'"
    return easyllm.generate_text(final_prd_prompt, max_tokens=DEFAULT_PRD_TOKENS)


def prd_generation(prompt: str) -> str:
    """Generates a PRD using the EasyLLM abstraction."""
    try:
        print(f"Generating PRD for: '{prompt}'")
        prd = generate_initial_prd(prompt)
        if not prd:
            raise Exception("Failed to generate PRD.")

        print(f"Generated PRD: {prd}")
        questions = generate_prd_questions(prompt, prd)
        if not questions:
            raise Exception("Failed to generate PRD questions.")

        print(f"Generated PRD questions: {questions}")
        answers = generate_prd_answers(prompt, prd, questions)
        if not answers:
            raise Exception("Failed to generate PRD answers.")

        print(f"Generated PRD answers: {answers}")
        final_prd = generate_final_prd(prompt, prd, questions, answers)
        if not final_prd:
            raise Exception("Failed to generate final PRD.")

        print(f"Generated final PRD: {final_prd}")
        return final_prd
    except Exception as e:
        print(f"Failed to generate PRD. Error: {str(e)}")
        traceback.print_exc()
        return "Failed to generate PRD."


In [ ]:
from IPython.display import Markdown, display, clear_output

prompt = "Instagram-like music sharing app for artists."

prd = prd_generation(prompt)

clear_output()
display(Markdown(prd))